# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

In [1]:
import sys
sys.path.append('./src/QA')
from importlib import reload
from data_prep import DataPrep
reload(data_prep)
from data_prep import DataPrep

ModuleNotFoundError: ignored

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [1]:
# only need to run these commands if using Colab
# Mount google drive and change directory to the git repo folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/CITS4012_2023

Mounted at /content/gdrive
/content/gdrive/MyDrive/CITS4012_2023


In [2]:
# colab seems to require an explicit path for this to work from mounted drive
import sys
sys.path.append('/content/gdrive/MyDrive/CITS4012_2023/src/QA')
from data_prep import DataPrep

In [3]:
# when not using colab
import sys
sys.path.append('./src/QA')
from data_prep import DataPrep

In [4]:
df = DataPrep.parse_tsv('WikiQA-train.tsv')
question_id_list = df.loc[:, "QuestionID"]
question_id_list = list(set(question_id_list))
question_id_list.sort() # fix sorting or dont sort

question_id = {}
for i in df.index:
    row = df.loc[i]
    id = row["QuestionID"]
    if id not in question_id:
        question_id[id] = {"document": []}
        question_id[id]["question"] = row["Question"]
    
    question_id[id]["document"].append((row["Sentence"], row["Label"]))

In [5]:
toks = DataPrep.tokenize_question_and_doc(question_id["Q1"])
toks

# currently DataPrep.tokenize_question_and_doc is returning a list of lists (one for each sentence), do we want to change that to one full list?

(['how', 'are', 'glacier', 'caves', 'formed?'],
 [[['A', 'OOA'],
   ['partly', 'OOA'],
   ['submerged', 'OOA'],
   ['glacier', 'OOA'],
   ['cave', 'OOA'],
   ['on', 'OOA'],
   ['Perito', 'OOA'],
   ['Moreno', 'OOA'],
   ['Glacier', 'OOA'],
   ['.', 'OOA']],
  [['The', 'OOA'],
   ['ice', 'OOA'],
   ['facade', 'OOA'],
   ['is', 'OOA'],
   ['approximately', 'OOA'],
   ['60', 'OOA'],
   ['m', 'OOA'],
   ['high', 'OOA']],
  [['Ice', 'OOA'],
   ['formations', 'OOA'],
   ['in', 'OOA'],
   ['the', 'OOA'],
   ['Titlis', 'OOA'],
   ['glacier', 'OOA'],
   ['cave', 'OOA']],
  [['A', 'BOA'],
   ['glacier', 'IOA'],
   ['cave', 'IOA'],
   ['is', 'IOA'],
   ['a', 'IOA'],
   ['cave', 'IOA'],
   ['formed', 'IOA'],
   ['within', 'IOA'],
   ['the', 'IOA'],
   ['ice', 'IOA'],
   ['of', 'IOA'],
   ['a', 'IOA'],
   ['glacier', 'IOA'],
   ['.', 'EOA']],
  [['Glacier', 'OOA'],
   ['caves', 'OOA'],
   ['are', 'OOA'],
   ['often', 'OOA'],
   ['called', 'OOA'],
   ['ice', 'OOA'],
   ['caves', 'OOA'],
   [',', 'OO

In [8]:
from word_embed import WordEmbed

[==============================================----] 93.0% 119.1/128.1MB downloaded


In [9]:
for sentence in toks:
  WordEmbed.glove_embed(toks[1][0])

a
partly
submerged
glacier
cave
on
perito
moreno
glacier
.


[array([-0.27086  ,  0.044006 , -0.02026  , -0.17395  ,  0.6444   ,
         0.71213  ,  0.3551   ,  0.47138  , -0.29637  ,  0.54427  ,
        -0.72294  , -0.0047612,  0.040611 ,  0.043236 ,  0.29729  ,
         0.10725  ,  0.40156  , -0.53662  ,  0.033382 ,  0.067396 ,
         0.64556  , -0.085523 ,  0.14103  ,  0.094539 ,  0.74947  ,
        -0.194    , -0.68739  , -0.41741  , -0.22807  ,  0.12     ,
        -0.48999  ,  0.80945  ,  0.045138 , -0.11898  ,  0.20161  ,
         0.39276  , -0.20121  ,  0.31354  ,  0.75304  ,  0.25907  ,
        -0.11566  , -0.029319 ,  0.93499  , -0.36067  ,  0.5242   ,
         0.23706  ,  0.52715  ,  0.22869  , -0.51958  , -0.79349  ,
        -0.20368  , -0.50187  ,  0.18748  ,  0.94282  , -0.44834  ,
        -3.6792   ,  0.044183 , -0.26751  ,  2.1997   ,  0.241    ,
        -0.033425 ,  0.69553  , -0.64472  , -0.0072277,  0.89575  ,
         0.20015  ,  0.46493  ,  0.61933  , -0.1066   ,  0.08691  ,
        -0.4623   ,  0.18262  , -0.15849  ,  0.0

# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 